In [40]:
#Import packages
import numpy as np
import pandas as pd

#Load test embeddings

test_connections = pd.read_csv('test_connections.csv')
targets = np.load('solution.npy')


In [42]:
# load people embedings
# Assume alphabetical
people_embeddings = np.load('people_embeddings.npy')

# load test connections embeddings
test_connections_embeddings = np.load('test_connections_embeddings.npy')

# load ip_to_person_id.csv
import pandas as pd

ip_to_person_id_new = pd.read_csv('ip_to_person_id.csv')

# cast pd to a map
#ip_to_person_id_map = ip_to_person_id_new['logins']

people = ip_to_person_id_new['logins']


val_to_get = 0

min = 999
index = 0
min_embedding = 0

correct_embedding_list = []
better_connection_embeddings = []



for outer_idx, test_connections_embedding in enumerate(test_connections_embeddings):

    test_connection = test_connections.loc[outer_idx]

    for idx, embedding in enumerate(people_embeddings):
        #take embedding minus first test connection embedding
        diff = embedding - test_connections_embedding

        #take the norm of the difference
        norm = np.linalg.norm(diff)

        #if norm is less than min, set min to norm
        if norm < min:
            min = norm
            index = idx
            min_embedding = diff
        
    predicted_person = people[index]

    

    #check if predicted person is the same as the true person
    
    if predicted_person == test_connections['logins'][outer_idx]:
        #print("Correct")
        correct_embedding_list.append(1)
    else:
        correct_embedding_list.append(0)
        #better_connection_embeddings.append(0)
        

    #print(people[index])
    # get test connections row at iloc outer_idx
    #print(test_connections.loc[outer_idx]['logins'])

    #print(min)
    #print(min_embedding)

    better_connection_embeddings.append(min_embedding)

    min = 999

correct_embedding_list = np.array(correct_embedding_list)

correct_embedding_bmap = correct_embedding_list.astype(bool)


In [43]:
#correct_embedding_bmap

In [44]:
# create a map of 

periods = []

for outer_idx, _ in enumerate(test_connections_embeddings):
    test_connection = test_connections.loc[outer_idx]

    # convert 09:53:31 03-07-2024 to an integer representing seconds

    dates = test_connection['dates'].split(' ')

    time = dates[0].split(':')

    seconds = int(time[0])*3600 + int(time[1])*60 + int(time[2])

    # convert 09:53:31 03-07-2024 to an integer representing days

    date = dates[1].split('-')

    days = int(date[0])*365 + int(date[1])*30 + int(date[2])

    # create seconds object

    seconds_obj = seconds + days * 24 * 3600


    periods.append([outer_idx, test_connection['logins'], seconds_obj, 1])

# create a pandas dataframe of periods

periods_df = pd.DataFrame(periods, columns=['index', 'logins', 'dates', 'true'])

# sort by logins and dates

periods_df = periods_df.sort_values(by=['logins', 'dates'])

periods_df.head()

,index,logins,dates,true
1781,1781,aaliyah_shawn@email.com,287661211,1
89,89,aaliyah_shawn@email.com,476892645,1
516,516,aaliyah_shawn@email.com,897231641,1
869,869,aarya_aditya@email.com,227195276,1
5636,5636,aarya_aditya@email.com,429765130,1


In [51]:
# interate through periods_df

tracker_login = ""
temp_login_delta_tracker = []
temp_ids = []


time_error = 0

for index, row in periods_df.iterrows():
    #print(index, row['logins'], row['dates'])

    current_login = row['logins']
    current_date = row['dates']

    if current_login != tracker_login:
        tracker_login = current_login
        temp_login_delta_tracker = [current_date]
        temp_ids = [index]
    
    else:
        temp_login_delta_tracker.append(current_date)
        temp_ids.append(index)

    if len(temp_login_delta_tracker) >= 3:
        first_delta = temp_login_delta_tracker[1] - temp_login_delta_tracker[0]
        second_delta = temp_login_delta_tracker[2] - temp_login_delta_tracker[1]

        # check if delta is the same plus or minus 5 seconds 
        # if so add 1 to time_error
        if abs(first_delta - second_delta) <= 1:
            time_error += 1
            
            # set row to 0
            periods_df.at[index, 'true'] = 0


            if len(temp_login_delta_tracker) == 3:
                time_error += 2

                # set previous two rows to 0
                periods_df.at[temp_ids[0], 'true'] = 0
                periods_df.at[temp_ids[1], 'true'] = 0


time_error

2563

In [52]:
periods_df.head()

,index,logins,dates,true
0,0,lara_goldy@email.com,1099854121,1
1,1,jaime_diego@email.com,855358615,1
2,2,frank_aria@email.com,245743926,1
3,3,leila_carmen@email.com,250935356,0
4,4,chace_maliyah@email.com,784482620,1


In [53]:
#sort periods df by index

periods_df = periods_df.sort_values(by=['index'])

# get a view of 4000 to 4100 of periods df

periods_df[4300:4700]

#make a binary map of the true column

true_map = periods_df['true'].values

# cast to numpy bools

true_map = true_map.astype(bool)

true_map

array([ True,  True,  True, ..., False, False, False])

In [54]:
true_map[:100]

array([ True,  True,  True, False,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True])

In [55]:
# targets and correct_list by logical equals
targets = np.load('solution.npy')

log_equal = targets == (true_map == correct_embedding_bmap)

#log_equal[4000:5000]

#correct_list[4000:5000]

#targets[4000:5000]

# count number of false in log equal

false_count = 0

for elem in log_equal:
    if elem == False:
        false_count += 1

false_count

949

In [56]:
targets = np.load('solution.npy')

def eval(predicted_test_labels):
    accuracy = np.sum(targets==predicted_test_labels)/len(predicted_test_labels)
    print(f"The accuracy of your system is {100*accuracy}%!")


